In [35]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated, operator
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [36]:
load_dotenv()

True

In [37]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [38]:
class EvaluationSchema(BaseModel):
    feedback:str = Field(description= "Detailed feedback for the essay")
    score:int = Field(description= "Score out of 10", ge=0, le =10)
    

In [39]:
structured_model = llm.with_structured_output(EvaluationSchema)

In [40]:
class UPSCState(TypedDict):
    essay:str
    thoughts_feedback: str
    depth_feedback: str
    language_feedback: str
    final_feedback: str
    final_score:int
    individual_score: Annotated[list[int], operator.add]
    


In [41]:
def thoughts_analysis(state: UPSCState):
    prompt = f'Evaluate the clarity of thoughts ,provide a feedback and give score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {"thoughts_feedback": output.feedback,"individual_score": [output.score] }


In [42]:
def depth_analysis(state: UPSCState):
    prompt = f'Evaluate the depth ,provide a feedback and give score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {"depth_feedback": output.feedback,"individual_score": [output.score] }

In [53]:
def language_analysis(state: UPSCState):
    prompt = f'Evaluate the analysis of language ,provide a feedback and give score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    return {"language_feedback": output.feedback,"individual_score": [output.score] }

In [52]:
def final_analysis(state: UPSCState):
    prompt = f"Based on following feedback, prepare a summarized feedback based on \n {state["thoughts_feedback"]}, {state["language_feedback"]} and{state["depth_feedback"]}"
    output1 = llm.invoke(prompt).content
    output2 = sum(state["individual_score"])/len(state["individual_score"])

    return {"final_feedback": output1, "final_score": output2}
    


In [54]:
# define graph
graph = StateGraph(UPSCState)

# add nodes
graph.add_node("thoughts_analysis", thoughts_analysis)
graph.add_node("depth_analysis", depth_analysis)
graph.add_node("language_analysis", language_analysis)
graph.add_node("final_analysis", final_analysis)

#add edges
graph.add_edge(START, "thoughts_analysis")
graph.add_edge(START, "depth_analysis")
graph.add_edge(START, "language_analysis")

graph.add_edge( "thoughts_analysis", "final_analysis")
graph.add_edge( "depth_analysis", "final_analysis")
graph.add_edge( "language_analysis", "final_analysis")

graph.add_edge( "final_analysis", END)

workflow = graph.compile()



In [55]:
essay = """India is a country with ancient history and vibrant traditions. It is home to many religions and languages which make it a unique nation. The Himalayas stand tall in the north while the seas surround the south. Great rivers flow through its plains and fertile lands feed millions of people.

India is proud of its democratic system where every citizen has the right to vote. The youth of the nation play a major role in shaping the future with their energy and ideas. The country has made progress in technology space exploration and education. Yet poverty and inequality continue to create challenges.

Indian culture is admired across the world. Yoga and Ayurveda bring health and balance. Festivals like Diwali and Holi bring joy and togetherness. Music dance and art showcase the creativity of the people.

In conclusion India is a land full of contrasts. It carries the wisdom of its past while stepping into the future with confidence. The journey of India shows how unity can survive in diversity and how hope can grow even with challenges."""

In [56]:
workflow.invoke( {"essay": essay})

{'essay': 'India is a country with ancient history and vibrant traditions. It is home to many religions and languages which make it a unique nation. The Himalayas stand tall in the north while the seas surround the south. Great rivers flow through its plains and fertile lands feed millions of people.\n\nIndia is proud of its democratic system where every citizen has the right to vote. The youth of the nation play a major role in shaping the future with their energy and ideas. The country has made progress in technology space exploration and education. Yet poverty and inequality continue to create challenges.\n\nIndian culture is admired across the world. Yoga and Ayurveda bring health and balance. Festivals like Diwali and Holi bring joy and togetherness. Music dance and art showcase the creativity of the people.\n\nIn conclusion India is a land full of contrasts. It carries the wisdom of its past while stepping into the future with confidence. The journey of India shows how unity can 